In [1]:
"""
REALISTIC RANGE + TP/SL OPTIMIZER
High-fidelity simulation with 1-min bars, slippage, commissions
Minimum 3:1 Risk/Reward ratio | 60 days historical data
"""
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta, date
import pytz
from collections import defaultdict, Counter

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"
CONTRACT_ID = "CON.F.US.MNQ.H26"

# Strategy parameters
POSITION_SIZE = 1

# Realism parameters
SLIPPAGE_POINTS = 0.25  # 1 tick for MNQ
COMMISSION_PER_CONTRACT = 0.62  # Per side (in/out = 2x)
POINT_VALUE = 0.5  # $0.50 per point for MNQ

# Grid search parameters
RANGE_DURATIONS = [60, 75, 90]  # minutes
RANGE_START_TIMES = [
    (9, 30), (9, 40), (9, 50), 
    (10, 0), (10, 10), (10, 20), (10, 30),
    (10, 40), (10, 50), (11, 0)
]

# TP/SL combinations to test (SL, TP) - all with R/R >= 3:1
TP_SL_COMBINATIONS = [
    # 3:1 R/R
    (3, 9), (5, 15), (7, 21), (10, 30), (15, 45),
    # 4:1 R/R
    (3, 12), (5, 20), (7, 28), (10, 40), (15, 60),
    # 5:1 R/R
    (3, 15), (5, 25), (7, 35), (10, 50), (15, 75),
    # 6:1 R/R
    (3, 18), (5, 30), (7, 42), (10, 60), (15, 90),
    # 8:1 R/R
    (3, 24), (5, 40), (7, 56), (10, 80),
    # 10:1 R/R
    (3, 30), (5, 50), (7, 70), (10, 100),
]

# Quality filters
MIN_TOTAL_TRADES = 15
MIN_WIN_RATE = 10.0

# ==================== API FUNCTIONS ====================
def authenticate():
    resp = requests.post(
        f"{API_BASE}/Auth/loginKey",
        json={"userName": USERNAME, "apiKey": KEY},
        headers={"accept": "text/plain", "Content-Type": "application/json"},
        timeout=15
    )
    return {
        "Authorization": f"Bearer {resp.json()['token']}",
        "accept": "text/plain",
        "Content-Type": "application/json"
    }

def fetch_bars(headers, unit, unit_number, days_back=70):
    end_time = datetime.now(pytz.UTC)
    start_time = end_time - timedelta(days=days_back)
    
    payload = {
        "contractId": CONTRACT_ID,
        "live": False,
        "startTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "endTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "unit": unit,
        "unitNumber": unit_number,
        "limit": 10000,
        "includePartialBar": True
    }
    
    resp = requests.post(f"{API_BASE}/History/retrieveBars", json=payload, headers=headers, timeout=30)
    return resp.json().get("bars", [])

def parse_bars_with_ny_time(bars):
    ny_tz = pytz.timezone('America/New_York')
    parsed = []
    
    for bar in bars:
        timestamp = bar['t'].replace('+00:00', 'Z')
        dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        dt_utc = pytz.utc.localize(dt_utc)
        dt_ny = dt_utc.astimezone(ny_tz)
        
        bar['dt_ny'] = dt_ny
        bar['date'] = dt_ny.date()
        parsed.append(bar)
    
    return parsed

def get_trading_days(bars_5m):
    """Get unique trading days from the data, excluding weekends and holidays"""
    
    # Major US market holidays to exclude
    EXCLUDED_HOLIDAYS = {
        # 2024 holidays
        date(2024, 1, 1), date(2024, 1, 15), date(2024, 2, 19),
        date(2024, 3, 29), date(2024, 5, 27), date(2024, 6, 19),
        date(2024, 7, 4), date(2024, 9, 2), date(2024, 11, 28),
        date(2024, 12, 25),
        # 2025 holidays
        date(2025, 1, 1), date(2025, 1, 20), date(2025, 2, 17),
        date(2025, 4, 18), date(2025, 5, 26), date(2025, 6, 19),
        date(2025, 7, 4), date(2025, 9, 1), date(2025, 11, 27),
        date(2025, 12, 25),
        # 2026 holidays
        date(2026, 1, 1), date(2026, 1, 19), date(2026, 2, 16),
        date(2026, 4, 3), date(2026, 5, 25), date(2026, 6, 19),
        date(2026, 7, 3), date(2026, 9, 7), date(2026, 11, 26),
        date(2026, 12, 25),
    }
    
    days = set()
    for bar in bars_5m:
        if bar['dt_ny'].hour >= 9 and (bar['dt_ny'].hour > 9 or bar['dt_ny'].minute >= 30):
            bar_date = bar['date']
            
            # Skip weekends
            if bar['dt_ny'].weekday() >= 5:
                continue
            
            # Skip holidays
            if bar_date in EXCLUDED_HOLIDAYS:
                continue
            
            days.add(bar_date)
    
    return sorted(list(days), reverse=True)[:60]

def get_range_for_window(bars_5m, target_date, start_hour, start_minute, duration_minutes):
    """Get the range high/low for a specific time window on a specific day"""
    end_minutes_total = start_hour * 60 + start_minute + duration_minutes
    end_hour = end_minutes_total // 60
    end_minute = end_minutes_total % 60
    
    range_bars = []
    for bar in bars_5m:
        if bar['date'] == target_date:
            bar_time_minutes = bar['dt_ny'].hour * 60 + bar['dt_ny'].minute
            start_time_minutes = start_hour * 60 + start_minute
            end_time_minutes = end_hour * 60 + end_minute
            
            if start_time_minutes <= bar_time_minutes < end_time_minutes:
                range_bars.append(bar)
    
    if not range_bars:
        return None
    
    range_high = max(bar['h'] for bar in range_bars)
    range_low = min(bar['l'] for bar in range_bars)
    
    return {
        'high': range_high,
        'low': range_low,
        'size': range_high - range_low
    }

def simulate_trade_realistic(direction, entry, stop, target, future_bars):
    """
    Realistic trade simulation with:
    - Slippage on entry/exit
    - Intra-bar logic when both SL and TP hit
    - Commissions
    """
    
    # Apply slippage to entry
    if direction == 'long':
        entry_with_slippage = entry + SLIPPAGE_POINTS
        stop_with_slippage = stop - SLIPPAGE_POINTS  # Worse stop
        target_with_slippage = target - SLIPPAGE_POINTS  # Worse target
    else:
        entry_with_slippage = entry - SLIPPAGE_POINTS
        stop_with_slippage = stop + SLIPPAGE_POINTS
        target_with_slippage = target + SLIPPAGE_POINTS
    
    for bar in future_bars:
        if direction == 'long':
            stop_hit = bar['l'] <= stop_with_slippage
            target_hit = bar['h'] >= target_with_slippage
            
            # If both hit in same bar, determine which was closer to entry
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                # Assume the closer one hit first (conservative)
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (stop_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2  # In and out
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (target_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
        
        else:  # SHORT
            stop_hit = bar['h'] >= stop_with_slippage
            target_hit = bar['l'] <= target_with_slippage
            
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (entry_with_slippage - stop_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (entry_with_slippage - target_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
    
    # Trade still open at end of day - close at market
    if future_bars:
        exit_price = future_bars[-1]['c']
        if direction == 'long':
            gross_pnl = (exit_price - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
        else:
            gross_pnl = (entry_with_slippage - exit_price) * POINT_VALUE * POSITION_SIZE
        
        commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
        net_pnl = gross_pnl - commissions
        return {'result': 'OPEN', 'pnl': net_pnl, 'exit_price': exit_price}
    
    return {'result': 'OPEN', 'pnl': 0, 'exit_price': entry_with_slippage}

def backtest_day_with_params(bars_5m, target_date, start_hour, start_minute, duration_minutes, stop_loss, take_profit):
    """Run backtest for a single day with specific range and TP/SL parameters"""
    
    # Get range data
    range_data = get_range_for_window(bars_5m, target_date, start_hour, start_minute, duration_minutes)
    if range_data is None:
        return None
    
    range_high = range_data['high']
    range_low = range_data['low']
    
    # Calculate when range ends
    end_minutes_total = start_hour * 60 + start_minute + duration_minutes
    end_hour = end_minutes_total // 60
    end_minute = end_minutes_total % 60
    
    # Get bars AFTER the range ends
    bars_after_range = []
    for bar in bars_5m:
        if bar['date'] == target_date:
            bar_time_minutes = bar['dt_ny'].hour * 60 + bar['dt_ny'].minute
            range_end_minutes = end_hour * 60 + end_minute
            
            if bar_time_minutes >= range_end_minutes:
                bars_after_range.append(bar)
    
    bars_after_range = list(reversed(bars_after_range))
    
    if len(bars_after_range) < 5:
        return None
    
    trades = []
    
    # Scan for setups
    for i in range(len(bars_after_range) - 1):
        current_bar = bars_after_range[i]
        next_bar = bars_after_range[i + 1]
        
        closed_above = current_bar['c'] > range_high
        closed_below = current_bar['c'] < range_low
        next_closed_inside = range_low <= next_bar['c'] <= range_high
        
        # SHORT SETUP
        if closed_above and next_closed_inside:
            entry_price = next_bar['c']
            stop_price = entry_price + stop_loss
            target_price = entry_price - take_profit
            
            future_bars = bars_after_range[i+2:]
            outcome = simulate_trade_realistic('short', entry_price, stop_price, target_price, future_bars)
            
            trades.append({
                'type': 'SHORT',
                'result': outcome['result'],
                'pnl': outcome['pnl']
            })
        
        # LONG SETUP
        elif closed_below and next_closed_inside:
            entry_price = next_bar['c']
            stop_price = entry_price - stop_loss
            target_price = entry_price + take_profit
            
            future_bars = bars_after_range[i+2:]
            outcome = simulate_trade_realistic('long', entry_price, stop_price, target_price, future_bars)
            
            trades.append({
                'type': 'LONG',
                'result': outcome['result'],
                'pnl': outcome['pnl']
            })
    
    return {
        'trades': trades,
        'range_size': range_data['size']
    }

# ==================== MAIN ANALYSIS ====================
def main():
    print("=" * 90)
    print("REALISTIC RANGE + TP/SL OPTIMIZER")
    print(f"5-min bars | Slippage: {SLIPPAGE_POINTS}pts | Commission: ${COMMISSION_PER_CONTRACT}/contract")
    print(f"Testing {len(RANGE_DURATIONS)} durations × {len(RANGE_START_TIMES)} starts × {len(TP_SL_COMBINATIONS)} TP/SL combos")
    print("=" * 90)
    
    print("\n[1/5] Authenticating...")
    headers = authenticate()
    print("✅ Authenticated")
    
    print("\n[2/5] Fetching 5-minute historical data...")
    bars_5m = fetch_bars(headers, unit=2, unit_number=5, days_back=70)
    print(f"✅ Fetched {len(bars_5m)} 5-minute bars")
    
    bars_5m = parse_bars_with_ny_time(bars_5m)
    
    print("\n[3/5] Identifying trading days...")
    trading_days = get_trading_days(bars_5m)
    print(f"✅ Found {len(trading_days)} valid trading days (weekdays only, excluding holidays)")
    print(f"   Date Range: {trading_days[-1]} to {trading_days[0]}")
    
    # Show day breakdown
    day_counts = Counter([d.strftime('%A') for d in trading_days])
    print(f"   Breakdown: Mon({day_counts.get('Monday', 0)}) Tue({day_counts.get('Tuesday', 0)}) "
          f"Wed({day_counts.get('Wednesday', 0)}) Thu({day_counts.get('Thursday', 0)}) Fri({day_counts.get('Friday', 0)})")
    
    weekend_count = sum(1 for d in trading_days if d.weekday() >= 5)
    if weekend_count > 0:
        print(f"   ⚠️ WARNING: {weekend_count} weekend days found!")
    else:
        print(f"   ✅ No weekends included")
    
    print("\n[4/5] Generating parameter combinations...")
    total_combinations = len(RANGE_DURATIONS) * len(RANGE_START_TIMES) * len(TP_SL_COMBINATIONS)
    print(f"✅ Testing {total_combinations} parameter combinations")
    
    print("\n[5/5] Running realistic grid search...")
    print("=" * 90)
    print("RUNNING OPTIMIZATION (this will take several minutes...)")
    print("=" * 90)
    
    # Store results for each combination
    all_results = []
    test_count = 0
    total_tests = total_combinations * len(trading_days)
    
    for duration in RANGE_DURATIONS:
        for start_hour, start_minute in RANGE_START_TIMES:
            for stop_loss, take_profit in TP_SL_COMBINATIONS:
                
                end_minutes_total = start_hour * 60 + start_minute + duration
                end_hour = end_minutes_total // 60
                end_minute = end_minutes_total % 60
                
                combo_key = f"{start_hour:02d}:{start_minute:02d}-{end_hour:02d}:{end_minute:02d} | SL:{stop_loss} TP:{take_profit}"
                
                stats = {
                    'total_trades': 0,
                    'wins': 0,
                    'losses': 0,
                    'open': 0,
                    'total_pnl': 0,
                    'total_win_pnl': 0,
                    'total_loss_pnl': 0,
                    'days_with_setups': 0,
                    'range_sizes': []
                }
                
                for day in trading_days:
                    test_count += 1
                    if test_count % 500 == 0:
                        print(f"Progress: {test_count}/{total_tests} tests ({100*test_count/total_tests:.1f}%)")
                    
                    result = backtest_day_with_params(
                        bars_5m, day, start_hour, start_minute, duration, stop_loss, take_profit
                    )
                    
                    if result is None:
                        continue
                    
                    trades = result['trades']
                    
                    if trades:
                        stats['days_with_setups'] += 1
                    
                    stats['total_trades'] += len(trades)
                    stats['range_sizes'].append(result['range_size'])
                    
                    for trade in trades:
                        stats['total_pnl'] += trade['pnl']
                        
                        if trade['result'] == 'WIN':
                            stats['wins'] += 1
                            stats['total_win_pnl'] += trade['pnl']
                        elif trade['result'] == 'LOSS':
                            stats['losses'] += 1
                            stats['total_loss_pnl'] += abs(trade['pnl'])
                        else:
                            stats['open'] += 1
                
                # Calculate metrics
                if stats['total_trades'] >= MIN_TOTAL_TRADES:
                    closed_trades = stats['wins'] + stats['losses']
                    win_rate = (stats['wins'] / closed_trades * 100) if closed_trades > 0 else 0
                    
                    if win_rate >= MIN_WIN_RATE:
                        profit_factor = (stats['total_win_pnl'] / stats['total_loss_pnl']) if stats['total_loss_pnl'] > 0 else float('inf')
                        expectancy = stats['total_pnl'] / stats['total_trades']
                        avg_range = sum(stats['range_sizes']) / len(stats['range_sizes']) if stats['range_sizes'] else 0
                        rr_ratio = take_profit / stop_loss
                        
                        all_results.append({
                            'combo': combo_key,
                            'window': f"{start_hour:02d}:{start_minute:02d}-{end_hour:02d}:{end_minute:02d}",
                            'duration': duration,
                            'sl': stop_loss,
                            'tp': take_profit,
                            'rr': rr_ratio,
                            'total_trades': stats['total_trades'],
                            'wins': stats['wins'],
                            'losses': stats['losses'],
                            'open': stats['open'],
                            'win_rate': win_rate,
                            'total_pnl': stats['total_pnl'],
                            'expectancy': expectancy,
                            'profit_factor': profit_factor,
                            'days_with_setups': stats['days_with_setups'],
                            'avg_range': avg_range
                        })
    
    print(f"\n✅ Completed {total_tests} backtests")
    print(f"✅ Found {len(all_results)} viable combinations (min {MIN_TOTAL_TRADES} trades, min {MIN_WIN_RATE}% win rate)")
    
    if not all_results:
        print("\n❌ No combinations met the minimum criteria!")
        return
    
    # Sort by different metrics
    by_pnl = sorted(all_results, key=lambda x: x['total_pnl'], reverse=True)
    by_expectancy = sorted(all_results, key=lambda x: x['expectancy'], reverse=True)
    by_profit_factor = sorted(all_results, key=lambda x: x['profit_factor'], reverse=True)
    by_win_rate = sorted(all_results, key=lambda x: x['win_rate'], reverse=True)
    
    # Display results
    print("\n" + "=" * 130)
    print("🏆 TOP 15 BY TOTAL P&L (with slippage & commissions)")
    print("=" * 130)
    print(f"{'Rank':<5}{'Window':<18}{'SL':<5}{'TP':<5}{'R:R':<6}{'Trades':<8}{'W/L':<10}{'Win%':<7}{'Total P&L':<12}{'Expect':<10}{'PF':<7}{'Days'}")
    print("-" * 130)
    
    for idx, r in enumerate(by_pnl[:15], 1):
        pf_str = f"{r['profit_factor']:.2f}" if r['profit_factor'] != float('inf') else "∞"
        print(f"{idx:<5}{r['window']:<18}{r['sl']:<5}{r['tp']:<5}{r['rr']:<6.1f}{r['total_trades']:<8}"
              f"{r['wins']}/{r['losses']:<8}{r['win_rate']:<6.1f}%${r['total_pnl']:<11.2f}"
              f"${r['expectancy']:<9.2f}{pf_str:<7}{r['days_with_setups']}")
    
    print("\n" + "=" * 130)
    print("💎 TOP 15 BY EXPECTANCY (Avg P&L per Trade)")
    print("=" * 130)
    print(f"{'Rank':<5}{'Window':<18}{'SL':<5}{'TP':<5}{'R:R':<6}{'Trades':<8}{'W/L':<10}{'Win%':<7}{'Total P&L':<12}{'Expect':<10}{'PF':<7}{'Days'}")
    print("-" * 130)
    
    for idx, r in enumerate(by_expectancy[:15], 1):
        pf_str = f"{r['profit_factor']:.2f}" if r['profit_factor'] != float('inf') else "∞"
        print(f"{idx:<5}{r['window']:<18}{r['sl']:<5}{r['tp']:<5}{r['rr']:<6.1f}{r['total_trades']:<8}"
              f"{r['wins']}/{r['losses']:<8}{r['win_rate']:<6.1f}%${r['total_pnl']:<11.2f}"
              f"${r['expectancy']:<9.2f}{pf_str:<7}{r['days_with_setups']}")
    
    print("\n" + "=" * 130)
    print("📊 TOP 15 BY PROFIT FACTOR")
    print("=" * 130)
    print(f"{'Rank':<5}{'Window':<18}{'SL':<5}{'TP':<5}{'R:R':<6}{'Trades':<8}{'W/L':<10}{'Win%':<7}{'Total P&L':<12}{'Expect':<10}{'PF':<7}{'Days'}")
    print("-" * 130)
    
    for idx, r in enumerate(by_profit_factor[:15], 1):
        pf_str = f"{r['profit_factor']:.2f}" if r['profit_factor'] != float('inf') else "∞"
        print(f"{idx:<5}{r['window']:<18}{r['sl']:<5}{r['tp']:<5}{r['rr']:<6.1f}{r['total_trades']:<8}"
              f"{r['wins']}/{r['losses']:<8}{r['win_rate']:<6.1f}%${r['total_pnl']:<11.2f}"
              f"${r['expectancy']:<9.2f}{pf_str:<7}{r['days_with_setups']}")
    
    print("\n" + "=" * 130)
    print("🎯 TOP 15 BY WIN RATE")
    print("=" * 130)
    print(f"{'Rank':<5}{'Window':<18}{'SL':<5}{'TP':<5}{'R:R':<6}{'Trades':<8}{'W/L':<10}{'Win%':<7}{'Total P&L':<12}{'Expect':<10}{'PF':<7}{'Days'}")
    print("-" * 130)
    
    for idx, r in enumerate(by_win_rate[:15], 1):
        pf_str = f"{r['profit_factor']:.2f}" if r['profit_factor'] != float('inf') else "∞"
        print(f"{idx:<5}{r['window']:<18}{r['sl']:<5}{r['tp']:<5}{r['rr']:<6.1f}{r['total_trades']:<8}"
              f"{r['wins']}/{r['losses']:<8}{r['win_rate']:<6.1f}%${r['total_pnl']:<11.2f}"
              f"${r['expectancy']:<9.2f}{pf_str:<7}{r['days_with_setups']}")
    
    # Show best overall
    best = by_pnl[0]
    print("\n" + "=" * 90)
    print("🏆 BEST OVERALL COMBINATION (by Total P&L)")
    print("=" * 90)
    print(f"\n⏰ Range Window: {best['window']} ({best['duration']} minutes)")
    print(f"💰 Stop Loss: {best['sl']} pts | Take Profit: {best['tp']} pts | R:R = {best['rr']:.1f}:1")
    print(f"\n📊 Performance (REALISTIC - includes slippage & commissions):")
    print(f"   Total Trades: {best['total_trades']}")
    print(f"   Wins: {best['wins']} | Losses: {best['losses']} | Open: {best['open']}")
    print(f"   Win Rate: {best['win_rate']:.1f}%")
    print(f"   Total P&L: ${best['total_pnl']:.2f}")
    print(f"   Expectancy: ${best['expectancy']:.2f} per trade")
    pf_str = f"{best['profit_factor']:.2f}" if best['profit_factor'] != float('inf') else "∞"
    print(f"   Profit Factor: {pf_str}")
    print(f"   Days with Setups: {best['days_with_setups']}/{len(trading_days)}")
    print(f"   Avg Range Size: {best['avg_range']:.1f} points")
    
    print("\n" + "=" * 90)
    print("✅ REALISTIC OPTIMIZATION COMPLETE")
    print("=" * 90)
    print("\nNote: These results include:")
    print(f"  • Slippage: {SLIPPAGE_POINTS} points per fill")
    print(f"  • Commissions: ${COMMISSION_PER_CONTRACT * 2:.2f} per contract round-trip")
    print(f"  • Intra-bar logic: Conservative exit when both SL/TP hit in same bar")
    print(f"  • 5-minute bars: Good balance of accuracy and data availability")

if __name__ == "__main__":
    main()

REALISTIC RANGE + TP/SL OPTIMIZER
5-min bars | Slippage: 0.25pts | Commission: $0.62/contract
Testing 3 durations × 10 starts × 28 TP/SL combos

[1/5] Authenticating...
✅ Authenticated

[2/5] Fetching 5-minute historical data...
✅ Fetched 9541 5-minute bars

[3/5] Identifying trading days...
✅ Found 34 valid trading days (weekdays only, excluding holidays)
   Date Range: 2025-12-15 to 2026-02-03
   Breakdown: Mon(7) Tue(8) Wed(7) Thu(5) Fri(7)
   ✅ No weekends included

[4/5] Generating parameter combinations...
✅ Testing 840 parameter combinations

[5/5] Running realistic grid search...
RUNNING OPTIMIZATION (this will take several minutes...)
Progress: 500/28560 tests (1.8%)
Progress: 1000/28560 tests (3.5%)
Progress: 1500/28560 tests (5.3%)
Progress: 2000/28560 tests (7.0%)
Progress: 2500/28560 tests (8.8%)
Progress: 3000/28560 tests (10.5%)
Progress: 3500/28560 tests (12.3%)
Progress: 4000/28560 tests (14.0%)
Progress: 4500/28560 tests (15.8%)
Progress: 5000/28560 tests (17.5%)
Prog